# Objective Functions

Specifying a problem instance is the first step toward using Forge's binary optimization quantum algorithms.

#### Declaring an objective function

Consider the function

$$
f(x_0, x_1, x_2) = 4 + x_0 + x_1 - x_2 + 3 x_0 x_1 + x_1 x_2
$$

where $x_0, x_1$, and $x_2$ are boolean variables which can be either 0 or 1. We encode $f$ as a `PolynomialObjective`.

In [ ]:
from qcware.types.optimization import PolynomialObjective, Constraints, BinaryProblem

# Keys designate variables in a term and values are coefficients.
polynomial = {
    (): 4,
    (0,): 1, 
    (1,): 1, 
    (2,): -1, 
    (0, 1): 3,
    (1, 2): 1, 
}

objective_function = PolynomialObjective(
    polynomial, 
    num_variables=3,
    domain='boolean'
)

print(objective_function)

Properties of our objective function can be read off:

In [ ]:
objective_function.num_variables

In [ ]:
# Polynomial degree of the objective (2 for a QUBO)
objective_function.degree

**Note**: `PolynomialObjective` requires that variables be enumerated as `0`, `1`, ..., `num_variables - 1`. See the following cells: 

In [ ]:
# # This raises an error because num_variables=3 but we refer to x_5 in the polynomial:
# objective_function = PolynomialObjective(
#     polynomial={(0, 5): 7}, 
#     num_variables=3,
#     domain='boolean'
# )

In [ ]:
# This does not raise an error although it contains superfluous variables (see below)
objective_function = PolynomialObjective(
    polynomial={(0, 5): 7}, 
    num_variables=6,
    domain='boolean'
)

#### Problem instances

Problem instances consist of:
1. An objective function
2. Optional [constraints](./constraints.ipynb)

These instances are the inputs for quantum optimization algorithms on Forge.

In [ ]:
problem_instance = BinaryProblem(
    objective=objective_function,
    constraints=None
)

print(problem_instance)

#### Simplification
Objective functions are automatically represented in a unique simplified form

In [ ]:
# (Note that x^2 is the same as x for a boolean variable x.)
polynomial = {(0, 0): 6}

objective_function = PolynomialObjective(
    polynomial, 
    num_variables=3,
    domain='boolean'
)

print(objective_function.polynomial)

#### Spin variables

Spin variables take on the values 1 and -1 instead of 0 and 1. To define such an objective function, use either `domain='spin'` or `domain=Domain.SPIN`:

In [ ]:
polynomial = {
    (0, 1): -2,
    (1, 2): 1,
    (1, 1): 5
}

objective_function = PolynomialObjective(
    polynomial=polynomial, 
    num_variables=3,
    domain='spin'
)

print(objective_function)

#### Computing values of objective functions

In [ ]:
# Define the objective
polynomial = {
    (0,): 1, 
    (1,): 1, 
    (2,): -1, 
    (0, 1): 3,
    (1, 2): 1, 
}
objective_function = PolynomialObjective(
    polynomial, 
    num_variables=3,
    domain='boolean'
)

# Select variable values
variable_assignment = {
    0: 1,
    1: 1,
    2: 0
}

# Compute value
objective_function.compute_value(variable_assignment)

#### Advanced usage: superfluous variables

A `PolynomialObjective` object tracks the number of variables you declare it to have even if it is constant in one or more of those variables. For example, the function
$$
f(x_0, x_1, x_2) = 10 x_2
$$
can be defined with

In [ ]:
objective_function = PolynomialObjective(
    polynomial={(2,): 10}, 
    num_variables=3,
)

print(objective_function)

In [ ]:
objective_function.num_variables

In [ ]:
# The number of variables that are not superfluous:
objective_function.num_active_variables

To fully reduce an objective function and eliminate these variables, use `reduce_variables`.

In [ ]:
objective_function.reduce_variables()

The way that variables are transfered from the original polynomial to the reduced one is encoded in `mapping`.

#### Advanced usage: converting to `qubovert`

[qubovert](./qubovert.ipynb) is a powerful tool for constructing and manipulating polynomials of binary variables. Objective functions in Forge can be constructed from `qubovert` objects and they can be converted back to them.

In [ ]:
# Start with an objective function
polynomial = {
    (0,): 1, 
    (1,): 1, 
    (2,): -1, 
    (0, 1): 3,
    (1, 2): 1, 
}
objective_function = PolynomialObjective(
    polynomial, 
    num_variables=3,
    domain='boolean'
)


# Get the corresponding qubovert object
pubo = objective_function.qubovert()

print('Created qubovert polynomial:')
print(pubo)

In [ ]:
# String names can also be used
pubo = objective_function.qubovert(use_variable_names=True)
print(pubo)

In [ ]:
# Convert to an equivalent polynomial with spin variables
converted = objective_function.qubovert_spin()

print('Converted to a spin polynomial:')
print(converted)